### Full POP model works

In [1]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using LinearAlgebra
using Dates
using HDF5

Consider simple Lindblad master equation with just one dissipator:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} e_1 & h_1 - i h_2 \\ h_1 + i h_2 & 0
   \end{pmatrix}
$

and as dissipator is traceless:

$
A = \begin{pmatrix} a_1 + i b_1 &   a_2 + i b_2\\  a_3 + i b_3 & -a_1 - i b_1
   \end{pmatrix} $
   
$\operatorname{Tr} A = 0$

In [2]:
#using DynamicPolynomials

@polyvar g

@polyvar e[1]
@polyvar h[1:2]

#Hˢʸᵐᵇ = [ 0.      0.
#         0.     e[1] ]

# Hˢʸᵐᵇₐₙ
Hˢʸᵐᵇ = [ e[1]               h[1] - im*h[2]
          h[1] + im*h[2]     0.             ] #fixed b

2×2 Matrix{Polynomial{true, ComplexF64}}:
 e₁                  h₁ + (0.0-1.0im)h₂
 h₁ + (0.0+1.0im)h₂  0.0+0.0im

In [28]:
@polyvar a[1:4]
@polyvar b[1:4]

#Aˢʸᵐᵇ = [  0.           a[2] + im*b[2]
#           0.           0.             ]

Aˢʸᵐᵇ = [ a[1]              a[2] + im*b[2]               #fixed phase
          a[3] + im*b[3]          -a[1]    ]

2×2 Matrix{Polynomial{true, Complex{Int64}}}:
 a₁              a₂ + (0+1im)b₂
 a₃ + (0+1im)b₃  -a₁

In [29]:
pwd()

"/home/zah/git/POP23"

In [30]:
parentdir = pwd()
data_dir = parentdir*"/DATA/"
println(data_dir)

output_directory = parentdir*"/MODELS/"

/home/zah/git/POP23/DATA/


"/home/zah/git/POP23/MODELS/"

### Train models loop

In [31]:
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γᵢ = "0.079477"

ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)

objₑₓ = 0
#objₑₓₐₙ = 0

@time for (ρ,t) in zip([ρᵍ, ρᵉ, ρˣ, ρʸ], [tᵍ, tᵉ, tˣ, tʸ])

    t = convert(Vector{Float64}, t)
    @assert maximum(diff(t)) ≈ minimum(diff(t)) ≈ t[2]-t[1]

    # Convert cut ρ series:
    ρ = convert(Vector{Matrix{ComplexF64}}, ρ[1:200])
    objₑₓ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])#, g)
    #objₑₓₐₙ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇₐₙ, [Aˢʸᵐᵇ])

end # of files (initial states) loop

  1.341548 seconds (9.19 M allocations: 1.180 GiB, 13.31% gc time)


In [32]:
solₑₓⁿᵉʷ, best_methodₑₓⁿᵉʷ = LiPoSID.sos_min_newton(objₑₓ) 

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.00011352393667407946
Global optimality certified with relative optimality gap 0.000005%!
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.00011344012757145145
The local solver failed refining the solution!
No higher TS step of the TSSOS hierarchy!
val_p = [0.0001134727077669595, 0.005216485060429932]


(PolyVar{true}[e₁, h₁, h₂, a₁, a₂, a₃, b₂, b₃] => [25.126132360560177, 0.00014654815310480505, 0.00019718984147728804, -0.0003058856513557143, 0.28157159548267224, 0.08019805680365935, 0.06287301157042789, 0.02351683048261518], "tssos")

In [22]:
best_methodₑₓⁿᵉʷ

"tssos"

In [33]:
Hˢⁱᵈₑₓ = subs(Hˢʸᵐᵇ, solₑₓⁿᵉʷ)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (25.1261+0.0im)             (0.000146548-0.00019719im)
 (0.000146548+0.00019719im)  0.0+0.0im

In [34]:
Hˢⁱᵈₑₓ = subs(Hˢʸᵐᵇ, solₑₓⁿᵉʷ)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (25.1261+0.0im)             (0.000146548-0.00019719im)
 (0.000146548+0.00019719im)  0.0+0.0im

In [35]:
Aˢⁱᵈₑₓ = subs(Aˢʸᵐᵇ, solₑₓⁿᵉʷ)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-0.000305886-0.0im)     (0.281572+0.062873im)
 (0.0801981+0.0235168im)  (0.000305886-0.0im)

In [19]:
Aˢⁱᵈₑₓ = subs(Aˢʸᵐᵇ, solₑₓⁿᵉʷ)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-0.000168331-0.000160678im)  (1.25496+1.34579im)
 (0.262725+0.321183im)         (0.000168331+0.000160678im)

In [29]:
gˢⁱᵈₑₓ = subs(g, solₑₓⁿᵉʷ)

-0.0032235046707173353

In [20]:
gˢⁱᵈₑₓ = subs(g, solₑₓⁿᵉʷ)

0.025182480044274872

In [23]:
α = 0.001 #regularization coefficient

robjₑₓ += α * (sum(a.^2)+sum(b.^2)) #adding regularization

rsolₑₓⁿᵉʷ, best_methodₑₓⁿᵉʷ = LiPoSID.sos_min_newton(robjₑₓ) 

Hˢⁱᵈₑₓ = subs(Hˢʸᵐᵇ, rsolₑₓⁿᵉʷ)
Aˢⁱᵈₑₓ = subs(Aˢʸᵐᵇ, rsolₑₓⁿᵉʷ)

gˢⁱᵈₑₓ = subs(g, rsolₑₓⁿᵉʷ)

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
 TSSOS failed
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
Scaled TSSOS failed
All methods fail !!!

LoadError: UndefVarError: start_time not defined